In [ ]:

from __future__ import print_function
from tensorflow import keras
import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn
import random
import collections
import time
import nltk
import re
from nltk.tokenize import sent_tokenize
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
with open("data.txt","r") as f:
    st=f.read()
f.close()
st=re.sub(r'(?:\n)+'," ",st)
st=re.sub(r'(?:-|_)+'," ",st)
stoken=sent_tokenize(st)

In [ ]:
from sklearn.model_selection import train_test_split
train_set,test_set=train_test_split(stoken,test_size=0.2,shuffle=False)
(train_set)
with open('train.txt', 'w') as lis:
    for i in train_set:
        lis.write('%s\n' % i)
lis.close()
with open('test.txt','w') as lis:
    for i in test_set:
        lis.write('%s\n' % i)
lis.close()

In [ ]:
train=[]
test=[]
with open('train1.txt', 'r') as lis:
    for i in lis:
        j = i[:-1]
        train.append(j)
lis.close()
with open('test1.txt', 'r') as lis:
    for i in lis:
        j = i[:-1]
        test.append(j)
lis.close()

In [ ]:
nTrain=[]
nTest=[]
for i in train:
    nTrain.extend(tknzr.tokenize(i))
for i in test:
    nTest.extend(tknzr.tokenize(i))
vocab=list(set(nTrain))
v=len(vocab)
print(v)

In [ ]:
from nltk.lm import MLE
from collections import defaultdict
from nltk.util import ngrams

In [244]:
#Unigram MLE
N=len(nTrain)
unigram_MLE=defaultdict(float)
for i in vocab:
    unigram_MLE[i]=1.0*(nTrain.count(i))/N
unigram_MLE

defaultdict(float,
            {'July': 7.564544475739245e-06,
             'faithfully': 1.0086059300985661e-05,
             'dishabille': 1.2607574126232076e-06,
             'pleases': 1.0086059300985661e-05,
             'LOVE': 2.5215148252464152e-06,
             'embarrassments': 3.7822722378696226e-06,
             'Adair': 1.2607574126232076e-06,
             'modestest': 1.2607574126232076e-06,
             'Mr': 0.0035162524238061257,
             'Imprudencies': 1.2607574126232076e-06,
             'toys': 2.5215148252464152e-06,
             'reflects': 2.5215148252464152e-06,
             'cucumber': 2.5215148252464152e-06,
             'fairly': 3.4040450140826604e-05,
             'confessed': 1.1346816713608868e-05,
             'flew': 1.3868331538855282e-05,
             'Ignoble': 1.2607574126232076e-06,
             'safely': 4.790878167968189e-05,
             'details': 5.0430296504928305e-06,
             'momentary': 2.7736663077710564e-05,
             'gown'

In [245]:
#Bigram MLE
cfd = nltk.ConditionalFreqDist(nltk.bigrams(nTrain))
bigram_MLE=defaultdict(float)
bigram_MLE={(a,b):1.0 *(cfd[a][b])/(cfd[a].N()) for (a,b) in nltk.bigrams(nTrain)} #v=len(vocab)
bigram_MLE

{('<s>', 'Chapter'): 0.0023587324376986977,
 ('Chapter', '1'): 0.028169014084507043,
 ('1', 'Sir'): 0.2,
 ('Sir', 'Walter'): 0.17729393468118196,
 ('Walter', 'Elliot'): 0.11290322580645161,
 ('Elliot', ','): 0.2964426877470356,
 (',', 'of'): 0.00818789963492097,
 ('of', 'Kellynch'): 0.0008144648965629581,
 ('Kellynch', 'Hall'): 0.3333333333333333,
 ('Hall', ','): 0.40540540540540543,
 (',', 'in'): 0.021609302129275962,
 ('in', 'Somersetshire'): 0.00021342439440828086,
 ('Somersetshire', ','): 1.0,
 (',', 'was'): 0.013780150674235549,
 ('was', 'a'): 0.05471515931766978,
 ('a', 'man'): 0.015772314741386906,
 ('man', 'who'): 0.07162921348314606,
 ('who', ','): 0.061142397425583264,
 (',', 'for'): 0.01477198387743474,
 ('for', 'his'): 0.02974247370330069,
 ('his', 'own'): 0.06333122229259025,
 ('own', 'amusement'): 0.004244482173174873,
 ('amusement', ','): 0.203125,
 (',', 'never'): 0.0006541878574292528,
 ('never', 'took'): 0.0025041736227045075,
 ('took', 'up'): 0.06280193236714976,
 ('

In [246]:
#Trigram MLE
cfd_tri = nltk.ConditionalFreqDist((((w0, w1), w2) for w0, w1, w2 in nltk.trigrams(nTrain)))
tri_MLE=defaultdict(float)
for i in (((w0, w1), w2) for w0, w1, w2 in nltk.trigrams(nTrain)):
    a,b=i
    tri_MLE[i]=(1.0 *(cfd_tri[a][b])/(cfd_tri[a].N()))
tri_MLE

defaultdict(float,
            {(('<s>', 'Chapter'), '1'): 0.014492753623188406,
             (('Chapter', '1'), 'Sir'): 0.5,
             (('1', 'Sir'), 'Walter'): 1.0,
             (('Sir', 'Walter'), 'Elliot'): 0.09649122807017543,
             (('Walter', 'Elliot'), ','): 0.35714285714285715,
             (('Elliot', ','), 'of'): 0.02666666666666667,
             ((',', 'of'), 'Kellynch'): 0.002577319587628866,
             (('of', 'Kellynch'), 'Hall'): 0.4666666666666667,
             (('Kellynch', 'Hall'), ','): 0.4166666666666667,
             (('Hall', ','), 'in'): 0.13333333333333333,
             ((',', 'in'), 'Somersetshire'): 0.0009765625,
             (('in', 'Somersetshire'), ','): 1.0,
             (('Somersetshire', ','), 'was'): 0.25,
             ((',', 'was'), 'a'): 0.05206738131699847,
             (('was', 'a'), 'man'): 0.00980392156862745,
             (('a', 'man'), 'who'): 0.11173184357541899,
             (('man', 'who'), ','): 0.0392156862745098,
             

In [247]:
#Quadgram MLE
cfd_quad = nltk.ConditionalFreqDist((((w0,w1,w2),w3) for w0,w1,w2,w3 in nltk.everygrams(nTrain, min_len=4, max_len=4)))
quad_MLE=defaultdict(float)
for i in (((w0,w1,w2), w3) for w0,w1,w2,w3 in nltk.everygrams(nTrain, min_len=4, max_len=4)):
    a,b=i
    quad_MLE[i]=(1.0 *(cfd_quad[a][b])/(cfd_quad[a].N()))
quad_MLE

defaultdict(float,
            {(('<s>', 'Chapter', '1'), 'Sir'): 1.0,
             (('Chapter', '1', 'Sir'), 'Walter'): 1.0,
             (('1', 'Sir', 'Walter'), 'Elliot'): 1.0,
             (('Sir', 'Walter', 'Elliot'), ','): 0.18181818181818182,
             (('Walter', 'Elliot', ','), 'of'): 0.2,
             (('Elliot', ',', 'of'), 'Kellynch'): 0.5,
             ((',', 'of', 'Kellynch'), 'Hall'): 1.0,
             (('of', 'Kellynch', 'Hall'), ','): 0.5714285714285714,
             (('Kellynch', 'Hall', ','), 'in'): 0.2,
             (('Hall', ',', 'in'), 'Somersetshire'): 0.5,
             ((',', 'in', 'Somersetshire'), ','): 1.0,
             (('in', 'Somersetshire', ','), 'was'): 0.5,
             (('Somersetshire', ',', 'was'), 'a'): 1.0,
             ((',', 'was', 'a'), 'man'): 0.029411764705882353,
             (('was', 'a', 'man'), 'who'): 0.2,
             (('a', 'man', 'who'), ','): 0.05,
             (('man', 'who', ','), 'for'): 0.5,
             (('who', ',', 'for'), '

In [ ]:
#Unigram
def Gen_uni():
    gen=defaultdict(str)
    string=["<s>"]
    for i in range(15):
        word,prob = random.choice(list(unigram_MLE.items()))
        gen[prob]=word
    key=sorted(gen.keys(),reverse=True)
    for i in key:
        string.append(gen[i])
    string.append("</s>")
    st=' '.join(string)
    return st
for i in range(5):
    sen=Gen_uni()
    print(sen,end="\n")

In [371]:
def Generator(n):
    string=["<s>"]
    if n==2:
        for k in range(5):
            string.append(random.choice(vocab))
            for j in range(20):
                t=(string[-1],)
                max=0
                for i in vocab:
                    new=(*t,i)
                    if new in bigram_MLE:
                        if max<bigram_MLE[new]:
                            max=bigram_MLE[new]
                            st=i
                if max>0 and st!="</s>":
                    string.append(st)
                elif st=="</s>":
                    string.extend(["</s>","<s>"])
                    break
                else:
                    return "string not possible"
                    break
            if string[-2]!="</s>":
                string.extend(["</s>","<s>"])
        return string
    elif n==3:
        for i in range(5):
            string.append(random.choice(vocab))
            for i in range(30):
                if string[-2]=="<s>":
                    t=(string[-1],)
                    max=0
                    for i in vocab:
                        new=(*t,i)
                        if new in bigram_MLE:
                            if max<bigram_MLE[new]:
                                max=bigram_MLE[new]
                                st=i
                    if max>0:
                        string.append(st)
                    elif st=="</s>":
                        string.extend(["</s>","<s>"])
                        break
                    else:
                        return "string not possible"
                        break
                else:
                    t=((string[-2],string[-1],),)
                    max=0
                    for i in vocab:
                        new=(*t,i)
                        if new in tri_MLE:
                            if max<tri_MLE[new]:
                                max=tri_MLE[new]
                                st=i
                    if max>0 and st!="</s>":
                        string.append(st)
                    elif st=="</s>":
                        string.extend(["</s>","<s>"])
                        break
                    else:
                        break
            if string[-2]!="</s>":
                string.extend(["</s>","<s>"])
        return string
    elif n==4:
        for j in range(5):
            string.append(random.choice(vocab))
            for i in range(50):
                if string[-2]=="<s>":
                    t=(string[-1],)
                    max=0
                    for i in vocab:
                        new=(*t,i)
                        if new in bigram_MLE:
                            if max<bigram_MLE[new]:
                                max=bigram_MLE[new]
                                st=i
                    if max>0:
                        string.append(st)
                    else:
                        return "string not possible"
                        break
                elif string[-3]=="<s>":
                    t=((string[-2],string[-1],),)
                    max=0
                    for i in vocab:
                        new=(*t,i)
                        if new in tri_MLE:
                            if max<tri_MLE[new]:
                                max=tri_MLE[new]
                                st=i
                    if max>0:
                        string.append(st)
                    elif st=="</s>":
                        string.extend(["</s>","<s>"])
                        break
                    else:
                        return "string not possible"
                        break
                else:
                    t=((string[-3],string[-2],string[-1],),)
                    max=0
                    for i in vocab:
                        new=(*t,i)
                        if new in quad_MLE:
                            if max<quad_MLE[new]:
                                max=quad_MLE[new]
                                st=i
                    if max>0 and st!="</s>":
                        string.append(st)
                    elif st=="</s>":
                        string.extend(["</s>","<s>"])
                        break
                    else:
                        break
            if string[-2]!="</s>":
                string.extend(["</s>","<s>"])
        return string
    elif n==1:
        for i in range(4):
            print(Gen_uni())
        return Gen_uni()

In [372]:
#Unigram
print(Generator(1))

<s> since aunt suggested apply pretensions doubting Happily placid ambitious Prices classed brawn </s>
<s> left answer receiving street provoking teasing repulsive Reader Servant Baldwin Powers governess's </s>
<s> visitor cry Exactly reformation surpassing Ormond </s>
<s> tete plagued repentance needle brighten </s>
<s> whether venture bewitching 28th erection enclosures </s>


In [373]:
#Bigram
print(Generator(2))

['<s>', 'Hush', '!', '</s>', '<s>', 'Harding', ')', ',', 'and', 'the', 'same', 'time', ',', 'and', 'the', 'same', 'time', ',', 'and', 'the', 'same', 'time', ',', 'and', 'the', 'same', '</s>', '<s>', 'Valancourt', 'when', 'she', 'had', 'been', 'a', 'very', 'much', 'to', 'be', 'a', 'very', 'much', 'to', 'be', 'a', 'very', 'much', 'to', 'be', 'a', '</s>', '<s>', 'remind', 'her', ',', 'and', 'the', 'same', 'time', ',', 'and', 'the', 'same', 'time', ',', 'and', 'the', 'same', 'time', ',', 'and', 'the', 'same', '</s>', '<s>', 'insurgents', ',', 'and', 'the', 'same', 'time', ',', 'and', 'the', 'same', 'time', ',', 'and', 'the', 'same', 'time', ',', 'and', 'the', 'same', 'time', '</s>', '<s>']


In [374]:
#Trigram
print(Generator(3))

['<s>', 'Mutual', 'enquiries', 'on', 'common', 'subjects', 'of', 'housekeeping', 'in', 'a', 'very', 'good', 'sort', 'of', 'thing', '.', '</s>', '<s>', 'detached', 'from', 'the', 'first', 'time', 'of', 'life', ',', 'and', 'the', 'two', 'gentlemen', 'came', 'in', ',', 'and', 'the', 'two', 'gentlemen', 'came', 'in', ',', 'and', 'the', 'two', 'gentlemen', 'came', 'in', ',', 'and', 'the', '</s>', '<s>', 'recovered', 'from', 'her', ',', 'and', 'the', 'two', 'gentlemen', 'came', 'in', ',', 'and', 'the', 'two', 'gentlemen', 'came', 'in', ',', 'and', 'the', 'two', 'gentlemen', 'came', 'in', ',', 'and', 'the', 'two', 'gentlemen', 'came', 'in', '</s>', '<s>', 'lessened', 'by', 'his', 'son', ',', 'and', 'the', 'two', 'gentlemen', 'came', 'in', ',', 'and', 'the', 'two', 'gentlemen', 'came', 'in', ',', 'and', 'the', 'two', 'gentlemen', 'came', 'in', ',', 'and', 'the', 'two', 'gentlemen', 'came', '</s>', '<s>', 'depending', 'on', 'the', 'subject', ',', 'and', 'the', 'two', 'gentlemen', 'came', 'in', 

In [375]:
#Quadgram
print(Generator(4))

['<s>', 'warm', ',', 'and', 'he', 'was', 'not', 'to', 'be', 'supposed', 'that', 'his', 'uncle', 'and', 'aunt', ',', 'and', 'see', 'how', 'it', 'is', ';', 'I', 'assure', 'you', ',', '"', 'said', 'he', ',', '"', 'I', 'am', 'very', 'glad', 'to', 'see', 'him', '.', '</s>', '<s>', 'removed', 'from', 'the', 'partialities', 'and', 'injustice', 'of', 'her', "father's", 'comfort', ',', 'she', 'was', 'very', 'sure', 'that', 'he', 'would', 'be', 'here', 'again', 'soon', ',', '"', 'said', 'he', ',', '"', 'I', 'am', 'very', 'glad', 'to', 'see', 'him', '.', '</s>', '<s>', 'mental', 'sorrow', 'have', 'certainly', 'no', 'necessary', 'proportions', '.', '</s>', '<s>', 'penetrate', 'this', 'mystery', ',', 'to', 'proceed', 'in', 'her', 'dressing', 'without', 'further', 'delay', '.', '"', '</s>', '<s>', 'lawyer', ',', 'I', 'believe', ',', 'in', 'every', 'other', 'respect', ',', 'in', 'consideration', 'of', 'her', ',', 'and', 'she', 'was', 'in', 'the', 'habit', 'of', 'such', 'general', 'publicity', 'and', 

In [414]:
#UnigramPerplexity
import math
uni_addOne=[]
vtest=set(nTest)
for i in vtest:
    t=(1+nTrain.count(i))/(N+v)
    if t==0:
        print("yes")
    uni_addOne.append(t)#v=len(vocab), vocab=set(nTrain)
uni_perplex=1
uni_addone=[math.log10(i) for i in uni_addOne]
print((1/uni_perplex)**(1/N))

In [ ]:
#BigramPerplexity
bi_addOne=[]
for i in nltk.bigrams(vtest):
    a,b=i
    bi_addOne.append(1.0*(1+cfd[a][b])/(cfd[a].N()+v))#v=len(vocab), vocab=set(nTrain)
import math
bi_perplex=1
bi_addone=[math.log10(i) for i in bi_addOne]
for i in bi_addone:
    bi_perplex=bi_perplex*i
print((bi_perplex)**(1/N))

In [ ]:
#TrigramPerplexity
tri_addOne=[]
tri_perplex=1
for i in (((w0, w1), w2) for w0, w1, w2 in nltk.trigrams(vTest)):
    a,b=i
    tri_addOne.append(1.0 *(1+cfd_tri[a][b])/(cfd_tri[a].N())+v)#v=len(vocab), vocab=set(nTrain)
import math
tri_perplex=1
tri_addone=[math.log10(i) for i in tri_addOne]
for i in tri_addone:
    tri_perplex=tri_perplex*i
print((1/tri_perplex)**(1/N))

In [ ]:
#QuadgramPerplexity
quad_addOne=[]
quad_perplex=1
for i in (((w0,w1,w2), w3) for w0,w1,w2,w3 in nltk.everygrams(vTest, min_len=4, max_len=4)):
    a,b=i
    quad_addOne.append(1.0 *(cfd_quad[a][b])/(cfd_quad[a].N())+v)#v=len(vocab), vocab=set(nTrain)
import math
quad_perplex=1
quad_addone=[math.log10(i) for i in quad_addOne]
for i in quad_addone:
    quad_perplex=quad_perplex*i
print((1/quad_perplex)**(1/N))